# 切木头最大价值问题

In [30]:
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30, 33]

In [31]:
from collections import defaultdict

In [32]:
price = defaultdict(int)

In [33]:
for i, p in enumerate(original_price): 
    price[i + 1] = p

In [35]:
price # 长度与价值的对应关系

defaultdict(int,
            {1: 1,
             2: 5,
             3: 8,
             4: 9,
             5: 10,
             6: 17,
             7: 17,
             8: 20,
             9: 24,
             10: 30,
             11: 33})

## Getting the max-price spliting by enumerate

In [26]:
def r(n):
    return max(
        [price[n]] + [r(i) + r(n-i) for i in range(1, n)]
    )

In [51]:
%time r(12)

Wall time: 142 ms


35

In [50]:
%time r(15)

Wall time: 4.08 s


43

指数级算法复杂度，后面变得非常慢

### 番外1：递归求解斐波那契数列的例子

In [14]:
"""
0 1 1 2 3 5 8 13 21 34 55

fib(0) = 0
fib(1) = 1
fib(2) = 0 + 1 = 1
fib(3) = fib(2) + fib(1) = 2
fib(4) = fib(3) + fib(2) = 2 + 1 = 3
fib(5) = fib(4) + fib(3) = 3 + 2 = 5
"""
def fib(n):
    if n <= 1: return n   # 第1，2个数就等于它们自己，n
    else:
        return fib(n-1) + fib(n-2)  # 从后面开始每一个数就等于前两个数之和

In [39]:
# 33 以后就开始有点慢了
%time fib(33)

Wall time: 1.4 s


3524578

### 番外2：一种更高效的fibonacci算法（带缓存器）

In [45]:
'''
使用cached缓存已经计算过的数
'''
cache = {}
def cached_fib(n):
    if n in cache: return cache[n]    # 如果n在缓存里，直接返回数值即可
    else:                             # 下面的话，其实和上面的算法一样，只不过，会把计算过的数放进缓存器而已
        if n <= 1:
            cache[n] = n
            return n
        else:
            result = fib(n-1) + fib(n-2)
            cache[n] = result
            return result

In [48]:
%time cached_fib(33)

Wall time: 1.47 s


3524578

## Analysis: How to optimize
## A Simpler Problem
#### We could make Fib Problem quick very easy!

In [52]:
from functools import wraps

In [53]:
"""
写一个装饰器，用来暂存
"""
def memo(f): 
    memo.already_computed = {}  
    # python 自带的 cache 有 lru_cache 缓存管理功能，会自动删除不常用的缓存，以释放内存空间
    @wraps(f)
    def _wrap(arg):
        result = None
        
        if arg in memo.already_computed: 
            result = memo.already_computed[arg]
        else:
            result = f(arg)
            memo.already_computed[arg] = result
        
        return result
    
    return _wrap

# Edit Distance 编辑距离

In [54]:
## Edit Distance

In [55]:
solution = {}

In [56]:
from functools import lru_cache

In [57]:
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  
        # string 1 delete tail
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  
        # string 1 add tail of string2
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    solution[(string1, string2)] = operation 
    
    return min_distance

In [59]:
edit_distance('ABCDE', 'ABCCEF')

2

In [61]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABC'): 'ADD C',
 ('A', 'ABCC'): 'ADD C',
 ('A', 'ABCCE'): 'ADD E',
 ('A', 'ABCCEF'): 'ADD F',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADD C',
 ('AB', 'ABCC'): 'ADD C',
 ('AB', 'ABCCE'): 'ADD E',
 ('AB', 'ABCCEF'): 'ADD F',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCC'): 'ADD C',
 ('ABC', 'ABCCE'): 'ADD E',
 ('ABC', 'ABCCEF'): 'ADD F',
 ('ABCD', 'A'): 'DEL D',
 ('ABCD', 'AB'): 'DEL D',
 ('ABCD', 'ABC'): 'DEL D',
 ('ABCD', 'ABCC'): 'SUB D => C',
 ('ABCD', 'ABCCE'): 'ADD E',
 ('ABCD', 'ABCCEF'): 'ADD F',
 ('ABCDE', 'A'): 'DEL E',
 ('ABCDE', 'AB'): 'DEL E',
 ('ABCDE', 'ABC'): 'DEL E',
 ('ABCDE', 'ABCC'): 'DEL E',
 ('ABCDE', 'ABCCE'): '',
 ('ABCDE', 'ABCCEF'): 'ADD F'}